## About the Author

Gerald M. Thomas is a Data Scientist and a graduate of Texas Tech University, Rawls College of Business Administration with a Master of Business Administration in Management and a Master of Science in Data Science. Mr. Thomas's professional career includes a public sector career in the national defense, finance, and economics industries.



## License

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

## Common Key Performing Indicators

Some common key performing indicators in the sales sector includes:
+  **Churn rate:**  The number of customers who quit doing business with a particular company in a given period.
+  **Average Order Value:** The dollar amount spent each time a customer places an order at the point of sale. A point of sale can be at a retail location or online. The formula for average order value is simply Gross Revenue divided by Total Number of Orders in a given period.
+  **Inventory Days:** The number of days to convert inventory into cash. Inventory days is calculated by using the Average Inventory Value on the balance sheet divided by Cost of Goods Sold multiplied by the number of days in a period.

## Common Table Expressions

A common table expression is a temporary table this is created, used, and discarded upon completion of the SQL. Common table expressions are useful options to prepare complicated reports by breaking larger SQL queries into smaller queries by using a WITH statement to define a common table expression. 

Common table expressions are comparable to classes in other programming languages because they often contain multiple sets of instructions (similar to functions). While most CTEs are non-recursive, they can also be recursive to define queries that refer to themselves (which can be useful for hierarchal data). Below is one example of a common table expression used to calculate churn rate as it relates to customer loyalty programs.

In [ ]:
WITH CustomerYearlyStats AS (
    -- Calculate yearly transaction count and average frequency for each customer
    SELECT
        t.customer_id,
        COUNT(DISTINCT t.transaction_date) AS distinct_purchase_days_last_year,
        CAST(COUNT(DISTINCT t.transaction_date) AS DECIMAL) / 365 AS avg_daily_frequency_last_year,
        COUNT(t.transaction_id) AS total_transactions_last_year -- Assuming transaction_id exists for count
    FROM
        transactions t
    WHERE
        t.transaction_date >= DATEADD(year, -1, GETDATE()) -- Last 12 months
        AND t.transaction_date < GETDATE()
    GROUP BY
        t.customer_id
    HAVING
        COUNT(t.transaction_id) >= 10 -- At least 10 transactions in the previous year
),

CustomerQuarterlyStats AS (
    -- Calculate quarterly distinct purchase days
    SELECT
        t.customer_id,
        COUNT(DISTINCT t.transaction_date) AS distinct_purchase_days_last_quarter,
        CAST(COUNT(DISTINCT t.transaction_date) AS DECIMAL) / 90 AS avg_daily_frequency_last_quarter -- Approx. 90 days in a quarter
    FROM
        transactions t
    WHERE
        t.transaction_date >= DATEADD(quarter, -1, GETDATE()) -- Last 3 months/quarter
        AND t.transaction_date < GETDATE()
    GROUP BY
        t.customer_id
),

CustomerSpendStats AS (
    -- Calculate total spend in the last 6 months
    SELECT
        t.customer_id,
        SUM(t.total_amount) AS total_spend_last_6_months
    FROM
        transactions t
    WHERE
        t.transaction_date >= DATEADD(month, -6, GETDATE()) -- Last 6 months
        AND t.transaction_date < GETDATE()
    GROUP BY
        t.customer_id
)

SELECT
    lys.customer_id,
    css.total_spend_last_6_months,
    lqs.distinct_purchase_days_last_quarter AS distinct_purchase_days_in_last_3_months
FROM
    CustomerYearlyStats lys
JOIN
    CustomerQuarterlyStats lqs ON lys.customer_id = lqs.customer_id
JOIN
    CustomerSpendStats css ON lys.customer_id = css.customer_id
JOIN
    loyalty_members lm ON lys.customer_id = lm.customer_id -- Ensure they are loyalty members
WHERE
    (CAST(lqs.avg_daily_frequency_last_quarter AS DECIMAL) / lys.avg_daily_frequency_last_year) < 0.8; -- Frequency decreased by > 20%

## Object Oriented Programming

While common table expressions are useful to perform complex, human readable queries. Object oriented programming can also perform similar tasks as Common Table Expressions. Similar to creating a CTE using a WITH statement in SQL, we can instantiate a class to create a data model and query data using SQL. 

In [ ]:
from datetime import datetime, timedelta
from collections import defaultdict
from decimal import Decimal

# --- 1. Data Models ---

class Transaction:
    """Represents a single customer transaction."""
    def __init__(self, transaction_id: int, customer_id: int, transaction_date: datetime, total_amount: Decimal):
        self.transaction_id = transaction_id
        self.customer_id = customer_id
        self.transaction_date = transaction_date
        self.total_amount = total_amount

    def __repr__(self):
        return f"Transaction(ID: {self.transaction_id}, Customer: {self.customer_id}, Date: {self.transaction_date.strftime('%Y-%m-%d')}, Amount: {self.total_amount})"

class LoyaltyMember:
    """Represents a loyalty program member."""
    def __init__(self, customer_id: int, member_since: datetime):
        self.customer_id = customer_id
        self.member_since = member_since

    def __repr__(self):
        return f"LoyaltyMember(Customer ID: {self.customer_id})"



## Building a Database in Python

Object oriented programming in Python can be useful to store information by using lists. A list in Python, in this case, transactions and loyalty members may be appended. This means we are able to add new transactions and add new loyalty members by creating a written set of instructions referred to as a function.

In [ ]:
class Database:
    """Simulates a database with transactions and loyalty members."""
    def __init__(self):
        self.transactions = []
        self.loyalty_members = []

    def add_transaction(self, transaction: Transaction):
        self.transactions.append(transaction)

    def add_loyalty_member(self, member: LoyaltyMember):
        self.loyalty_members.append(member)

    def get_transactions(self):
        return self.transactions

    def get_loyalty_members(self):
        return self.loyalty_members

Using the data enterred into our data model, object oriented programming allows users to run analytics by creating reusable reports. A common concern for loyalty programs is the "churn" rate. Churn refers to customers who stop doing business with a company in a given period. To identify at risk customers who churn, object oriented programming can use a function to identify transactions in a given period, calculate total customer transactions in a given year, calculate total transactions in a given quarter, and identify customer spending habits in a 6 month period.

In [ ]:
class CustomerAnalytics:
    """
    Performs the analytics calculations similar to the SQL query.
    Encapsulates the logic for deriving customer insights.
    """
    def __init__(self, transactions: list[Transaction], loyalty_members: list[LoyaltyMember]):
        self.transactions = transactions
        self.loyalty_members = loyalty_members
        self.now = datetime.now() # Equivalent to GETDATE()

    def _get_transactions_in_period(self, start_date: datetime, end_date: datetime):
        """Helper to filter transactions within a given date range."""
        return [
            t for t in self.transactions
            if start_date <= t.transaction_date < end_date
        ]

    def calculate_customer_yearly_stats(self):
        """
        Calculates distinct purchase days, avg daily frequency, and total transactions
        for the last year, for customers with at least 10 transactions.
        Equivalent to CustomerYearlyStats CTE.
        """
        one_year_ago = self.now - timedelta(days=365)
        yearly_transactions = self._get_transactions_in_period(one_year_ago, self.now)

        customer_data = defaultdict(lambda: {'distinct_dates': set(), 'total_transactions': 0})
        for t in yearly_transactions:
            customer_data[t.customer_id]['distinct_dates'].add(t.transaction_date.date())
            customer_data[t.customer_id]['total_transactions'] += 1

        yearly_stats = {}
        for customer_id, data in customer_data.items():
            total_transactions = data['total_transactions']
            if total_transactions >= 10:  # HAVING COUNT(t.transaction_id) >= 10
                distinct_purchase_days = len(data['distinct_dates'])
                avg_daily_frequency = Decimal(distinct_purchase_days) / Decimal(365)
                yearly_stats[customer_id] = {
                    'distinct_purchase_days_last_year': distinct_purchase_days,
                    'avg_daily_frequency_last_year': avg_daily_frequency,
                    'total_transactions_last_year': total_transactions
                }
        return yearly_stats

    def calculate_customer_quarterly_stats(self):
        """
        Calculates distinct purchase days and avg daily frequency for the last quarter.
        Equivalent to CustomerQuarterlyStats CTE.
        """
        three_months_ago = self.now - timedelta(days=90) # Approx. 90 days for quarter
        quarterly_transactions = self._get_transactions_in_period(three_months_ago, self.now)

        customer_data = defaultdict(lambda: {'distinct_dates': set()})
        for t in quarterly_transactions:
            customer_data[t.customer_id]['distinct_dates'].add(t.transaction_date.date())

        quarterly_stats = {}
        for customer_id, data in customer_data.items():
            distinct_purchase_days = len(data['distinct_dates'])
            avg_daily_frequency = Decimal(distinct_purchase_days) / Decimal(90)
            quarterly_stats[customer_id] = {
                'distinct_purchase_days_last_quarter': distinct_purchase_days,
                'avg_daily_frequency_last_quarter': avg_daily_frequency
            }
        return quarterly_stats

    def calculate_customer_spend_stats(self):
        """
        Calculates total spend for the last 6 months.
        Equivalent to CustomerSpendStats CTE.
        """
        six_months_ago = self.now - timedelta(days=180) # Approx. 180 days for 6 months
        six_month_transactions = self._get_transactions_in_period(six_months_ago, self.now)

        customer_spend = defaultdict(Decimal)
        for t in six_month_transactions:
            customer_spend[t.customer_id] += t.total_amount

        spend_stats = {}
        for customer_id, total_spend in customer_spend.items():
            spend_stats[customer_id] = {'total_spend_last_6_months': total_spend}
        return spend_stats

    def get_at_risk_loyalty_members(self):
        """
        Combines all calculated stats and applies the final filtering logic.
        Equivalent to the final SELECT and JOINs.
        """
        yearly_stats = self.calculate_customer_yearly_stats()
        quarterly_stats = self.calculate_customer_quarterly_stats()
        spend_stats = self.calculate_customer_spend_stats()

        # Convert loyalty members to a set for efficient lookup
        loyalty_member_ids = {lm.customer_id for lm in self.loyalty_members}

        results = []
        # Simulate JOINs by iterating through the most restrictive set (yearly_stats, due to HAVING clause)
        for customer_id, lys_data in yearly_stats.items():
            # Check if customer is a loyalty member and has data in other stats
            if (customer_id in loyalty_member_ids and
                customer_id in quarterly_stats and
                customer_id in spend_stats):

                lqs_data = quarterly_stats[customer_id]
                css_data = spend_stats[customer_id]

                # Apply the final WHERE condition
                # (CAST(lqs.avg_daily_frequency_last_quarter AS DECIMAL) / lys.avg_daily_frequency_last_year) < 0.8;
                if lys_data['avg_daily_frequency_last_year'] > 0: # Avoid division by zero
                    if (lqs_data['avg_daily_frequency_last_quarter'] / lys_data['avg_daily_frequency_last_year']) < Decimal('0.8'):
                        results.append({
                            'customer_id': customer_id,
                            'total_spend_last_6_months': css_data['total_spend_last_6_months'],
                            'distinct_purchase_days_in_last_3_months': lqs_data['distinct_purchase_days_last_quarter']
                        })
        return results